# Fine-tuning Llama 3 for Reasoning with QLoRA (Drive-Integrated)

This notebook demonstrates fine-tuning Llama 3 8B using QLoRA for improved reasoning capabilities, with all data saved to Google Drive for persistence.

## 1. Setup and Installation

First, check GPU availability, install dependencies, and set up persistent storage.

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Mount Google Drive for persistent storage
from google.colab import drive
drive.mount('/content/drive')

# Create project directories in Drive
!mkdir -p /content/drive/MyDrive/llm-trainer-output/datasets
!mkdir -p /content/drive/MyDrive/llm-trainer-output/models
!mkdir -p /content/drive/MyDrive/llm-trainer-output/evaluation

In [ ]:
# Clone the repository
!git clone https://github.com/vmm/llm-trainer.git
%cd llm-trainer

In [ ]:
# Install dependencies
!pip install -r requirements.txt

In [ ]:
# Fix module import issues
import os
import sys

# Check and fix the working directory
if not os.path.exists('src'):
    # If we're not in the repo root, try to find it
    if os.path.exists('llm-trainer'):
        %cd llm-trainer
    else:
        # If we can't find it, raise an error
        raise FileNotFoundError("Cannot find repository root directory with 'src' folder")

# Add the current directory to Python's path
sys.path.append('.')
print(f"Working directory: {os.getcwd()}")
print(f"Python path includes current directory: {'./' in sys.path or '.' in sys.path}")

In [ ]:
# Set up periodic saves to Google Drive
import time
import threading

def save_checkpoint_periodically(interval=1800):  # 1800 seconds = 30 minutes
    while True:
        time.sleep(interval)
        print("\nSaving checkpoint to Google Drive...")
        # Synchronize any files that might have changed
        !mkdir -p output 2>/dev/null || true
        !cp -r output/* /content/drive/MyDrive/llm-trainer-output/models/ 2>/dev/null || true
        !cp -r data/* /content/drive/MyDrive/llm-trainer-output/datasets/ 2>/dev/null || true
        print(f"Checkpoint saved at {time.strftime('%H:%M:%S')}")

# Start the checkpoint thread
checkpoint_thread = threading.Thread(target=save_checkpoint_periodically, daemon=True)
checkpoint_thread.start()
print("Automatic checkpointing to Drive enabled (every 30 minutes)")

## 2. Create Drive-Integrated Configuration

Update the configuration to save outputs to Google Drive.

In [ ]:
# Update training config to save to Drive
import yaml

with open('configs/llama3_reasoning.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Update output directory
config['training']['output_dir'] = "/content/drive/MyDrive/llm-trainer-output/models/llama3_reasoning"

# Save updated config
with open('configs/llama3_reasoning_drive.yaml', 'w') as f:
    yaml.dump(config, f)

print("Updated config saved to configs/llama3_reasoning_drive.yaml")

## 3. Authenticate and Process Data

Authenticate with Hugging Face to access the gated Llama 3 model, then process the dataset.

In [ ]:
# Authenticate with Hugging Face
import os
from huggingface_hub import login

# Replace with your actual token
HF_TOKEN = "your_huggingface_token_here"  

# Log in to Hugging Face
login(token=HF_TOKEN)

# Set environment variable for other libraries
os.environ["HUGGING_FACE_HUB_TOKEN"] = HF_TOKEN
os.environ["HF_TOKEN"] = HF_TOKEN

In [ ]:
# Define drive paths for datasets
drive_dataset_path = "/content/drive/MyDrive/llm-trainer-output/datasets/natural_reasoning_processed"

# Check if dataset already exists in Drive
import os
if os.path.exists(drive_dataset_path):
    print(f"Dataset already exists at {drive_dataset_path}")
    # Create a symlink to local directory for easier access
    !mkdir -p data
    !ln -sf {drive_dataset_path} data/natural_reasoning_processed
else:
    # Process the dataset and save directly to Drive
    print("Processing dataset and saving to Drive...")
    !python -m src.data_processors.reasoning_processor --config configs/llama3_reasoning.yaml --output_path {drive_dataset_path}

In [ ]:
# Verify dataset structure
from datasets import load_from_disk

# Load the processed dataset
dataset = load_from_disk(drive_dataset_path)

# Print info about the dataset
print(f"Dataset splits: {dataset.keys()}")
if 'train' in dataset:
    print(f"Train size: {len(dataset['train'])}")
if 'validation' in dataset:
    print(f"Validation size: {len(dataset['validation'])}")

# See the first example
print("\nExample data:")
print(dataset[list(dataset.keys())[0]][0])

## 4. Fine-tuning with QLoRA

Fine-tune the Llama 3 model using QLoRA with all outputs saved to Drive.

In [ ]:
# Define model output path
drive_model_path = "/content/drive/MyDrive/llm-trainer-output/models/llama3_reasoning"

# Check if fine-tuned model already exists
import os
if os.path.exists(os.path.join(drive_model_path, "adapter_model")):
    print(f"Fine-tuned model already exists at {drive_model_path}/adapter_model")
    print("Skipping training step. If you want to retrain, delete this directory from your Drive.")
else:
    # Fine-tune the model
    print("Starting fine-tuning process (this may take several hours)...")
    !python -m src.trainers.qlora_trainer configs/llama3_reasoning_drive.yaml --dataset_path {drive_dataset_path}

## 5. Evaluation

Evaluate the fine-tuned model on the LogiQA benchmark.

In [ ]:
# Define paths for evaluation
eval_output_dir = "/content/drive/MyDrive/llm-trainer-output/evaluation/reasoning_results"

# Create directory if it doesn't exist
!mkdir -p {eval_output_dir}

# Evaluate the model
!python -m src.evaluators.reasoning_evaluator --config configs/llama3_reasoning_drive.yaml \
    --model_path {drive_model_path} \
    --output_dir {eval_output_dir}

## 6. Compare Models

Compare the performance of the base model vs. the fine-tuned model.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Try to load actual results from evaluation
results_path = os.path.join(eval_output_dir, "Meta-Llama-3-8B_results.txt")
finetuned_results = {"accuracy": 0.75}  # Default if file doesn't exist

if os.path.exists(results_path):
    with open(results_path, 'r') as f:
        for line in f:
            if line.startswith("accuracy"):
                finetuned_results["accuracy"] = float(line.split(":")[1].strip())
    print(f"Loaded actual evaluation results: {finetuned_results}")
else:
    print("Using placeholder results - actual evaluation results not found")

# Base model results (placeholder - replace with actual if available)
base_model_results = {"accuracy": 0.65}

# Create comparison dataframe
df = pd.DataFrame({
    "Model": ["Base Llama 3 8B", "Fine-tuned Llama 3 8B"],
    "Accuracy": [base_model_results["accuracy"], finetuned_results["accuracy"]]
})

# Plot comparison
plt.figure(figsize=(10, 6))
ax = df.plot.bar(x="Model", y="Accuracy", rot=0)
ax.set_ylim(0, 1.0)
ax.set_title("Reasoning Performance Comparison")

for i, v in enumerate(df["Accuracy"]):
    ax.text(i, v + 0.02, f"{v:.2f}", ha="center")

plt.tight_layout()
plot_path = os.path.join(eval_output_dir, "model_comparison.png")
plt.savefig(plot_path)
plt.show()

print(f"Comparison plot saved to {plot_path}")

## 7. Package LoRA Adapter for Download

Create a downloadable package of the adapter for later use.

In [ ]:
# Define paths
adapter_path = os.path.join(drive_model_path, "adapter_model")
export_path = "/content/drive/MyDrive/llm-trainer-output/lora_adapter"
zip_path = "/content/drive/MyDrive/llm-trainer-output/lora_adapter.zip"

if os.path.exists(adapter_path):
    # Create export directory
    !mkdir -p {export_path}
    
    # Copy adapter files
    !cp -r {adapter_path}/* {export_path}/
    
    print(f"Adapter exported to {export_path}")
    
    # Create a zip file for easy download
    !cd /content/drive/MyDrive/llm-trainer-output && zip -r lora_adapter.zip lora_adapter
    print(f"Adapter ZIP file created at {zip_path}")
    
    # Display file sizes
    !du -h {export_path} {zip_path}
else:
    print(f"Adapter not found at {adapter_path}")

## 8. Test the Fine-tuned Model

Try out the fine-tuned model on custom reasoning questions.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel, PeftConfig

# Load the adapter config
config = PeftConfig.from_pretrained(drive_model_path)

# Load base model with authentication
base_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    device_map="auto",
    trust_remote_code=True,
    token=HF_TOKEN
)

# Load adapter model
model = PeftModel.from_pretrained(base_model, drive_model_path, is_trainable=False)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    config.base_model_name_or_path, 
    trust_remote_code=True,
    token=HF_TOKEN
)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Create text generation pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

In [ ]:
# Test on some custom questions
test_questions = [
    "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly?",
    "If no mammals can fly, and all bats can fly, what can we conclude about bats?",
    "If all A are B, and all B are C, what can we conclude about the relationship between A and C?"
]

# Create a file to store results
test_results_path = os.path.join(eval_output_dir, "custom_test_results.txt")
with open(test_results_path, "w") as f:
    for question in test_questions:
        prompt = f"Question: {question}\n\nAnswer: "
        result = pipe(prompt, return_full_text=False)[0]["generated_text"]
        
        print(f"Question: {question}")
        print(f"Answer: {result}")
        print("-" * 80)
        
        # Also write to file
        f.write(f"Question: {question}\n")
        f.write(f"Answer: {result}\n")
        f.write("-" * 80 + "\n\n")

print(f"Test results also saved to {test_results_path}")

## 9. Access Your Outputs After Colab Shutdown

All important files are now stored in your Google Drive and will persist even after the Colab session ends. Here's how to find them:

1. **Processed Dataset**: `/content/drive/MyDrive/llm-trainer-output/datasets/natural_reasoning_processed`
2. **Fine-tuned Model**: `/content/drive/MyDrive/llm-trainer-output/models/llama3_reasoning`
3. **LoRA Adapter**: `/content/drive/MyDrive/llm-trainer-output/lora_adapter` and `lora_adapter.zip`
4. **Evaluation Results**: `/content/drive/MyDrive/llm-trainer-output/evaluation/reasoning_results`

You can visit these directories in your Google Drive web interface or using the file browser in a new Colab session.

In [ ]:
# List all saved files in Drive
!find /content/drive/MyDrive/llm-trainer-output -type d | sort

In [ ]:
# Display a summary of what was created
print("=== LLM Fine-tuning Summary ===")
print(f"Dataset: {os.path.exists(drive_dataset_path)}")
print(f"Trained Model: {os.path.exists(drive_model_path)}")
print(f"LoRA Adapter: {os.path.exists(export_path)}")
print(f"Evaluation Results: {os.path.exists(eval_output_dir)}")
print("\nAll files are stored in your Google Drive and will be available after this Colab session ends.")